In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import expon, weibull_min, norm, lognorm, kstest, ks_1samp, normaltest, skew, kurtosis
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# INPUT DATA

In [ ]:
#Input data Time To Failure

ttf_data = np.array([
    3107.5,
    7677.25,
    925,
    2751,
    8567.67,
    7156,
    4390.5,
    5422.5,
    4335.5,
    4307.5,
    11901.13,
    6985.5,
    5373.5
])

# MTTF/MTTR

## ANALISIS DISTRIBUSI

In [ ]:
#Statistik Data

ttf_df = pd.DataFrame(ttf_data)
ttf_df.describe()

,0
count,13.000000
mean,5607.734615
std,2860.163721
min,925.000000
25%,4307.500000
50%,5373.500000
75%,7156.000000
max,11901.130000


In [ ]:
# Fungsi untuk menguji kecocokan distribusi
def goodness_of_fit(ttf_data, dist, dist_name):
    params = dist.fit(ttf_data)
    log_likelihood = np.sum(dist.logpdf(ttf_data, *params))
    aic = 2 * len(params) - 2 * log_likelihood
    bic = len(params) * np.log(len(ttf_data)) - 2 * log_likelihood
    ks_stat, p_value = ks_1samp(ttf_data, lambda x: dist.cdf(x, *params))
    return dist_name, log_likelihood, aic, bic, ks_stat, p_value, params

# Hitung skewness dan kurtosis
skewness = skew(ttf_data)
kurt = kurtosis(ttf_data)

# Uji distribusi
normal_p = normaltest(ttf_data).pvalue
dist_results = {}
params_dict = {}

# Cek distribusi terbaik
for dist, name in [(expon, "Exponential"), (weibull_min, "Weibull"), (lognorm, "Log-Normal"), (norm, "Normal")]:
    dist_name, log_likelihood, aic, bic, ks_stat, p_value, params = goodness_of_fit(ttf_data, dist, name)
    dist_results[dist_name] = (log_likelihood, aic, bic, ks_stat, p_value)
    params_dict[dist_name] = params

# Menentukan distribusi terbaik berdasarkan AIC dan KS Test
best_fit_aic = min(dist_results, key=lambda x: dist_results[x][1])  # AIC lebih rendah lebih baik
best_fit_ks = min(dist_results, key=lambda x: dist_results[x][3])  # KS Statistic lebih rendah lebih baik

# Cetak hasil uji distribusi
print("\n=== Analisis Distribusi ===")
print(f"Skewness: {skewness:.2f}, Kurtosis: {kurt:.2f}")

for dist_name, (log_likelihood, aic, bic, ks_stat, p_value) in dist_results.items():
    print(f"\n=== {dist_name} ===")
    print(f"Log-Likelihood: {log_likelihood:.2f}")
    print(f"AIC: {aic:.2f}, BIC: {bic:.2f}")
    print(f"KS Stat: {ks_stat:.2f}, p-value: {p_value:.3f}")

print("\n=== Distribusi Terbaik ===")
#print(f"\nBest Fit (AIC): {best_fit_aic}")
print(f"Best Fit (KS Stat): {best_fit_ks}")


=== Analisis Distribusi ===
Skewness: 0.53, Kurtosis: 0.06

=== Exponential ===
Log-Likelihood: -122.87
AIC: 249.74, BIC: 250.87
KS Stat: 0.28, p-value: 0.203

=== Weibull ===
Log-Likelihood: -120.95
AIC: 247.90, BIC: 249.60
KS Stat: 0.13, p-value: 0.971

=== Log-Normal ===
Log-Likelihood: -119.76
AIC: 245.51, BIC: 247.21
KS Stat: 0.50, p-value: 0.001

=== Normal ===
Log-Likelihood: -121.39
AIC: 246.78, BIC: 247.91
KS Stat: 0.14, p-value: 0.922

=== Distribusi Terbaik ===
Best Fit (KS Stat): Weibull


In [ ]:
# === Estimasi Parameter ===
# Eksponensial
expon_loc_ttf, expon_scale_ttf = expon.fit(ttf_data, floc=0)

# Weibull
weibull_shape_ttf, weibull_loc_ttf, weibull_scale_ttf = weibull_min.fit(ttf_data, floc=0)

# Normal
normal_mean_ttf, normal_std_ttf = norm.fit(ttf_data)

# Lognormal
lognormal_shape_ttf, lognormal_loc_ttf, lognormal_scale_ttf = lognorm.fit(ttf_data, floc=0)

print("=== Parameter Estimasi ===")
print(f"Exponential: loc={expon_loc_ttf}, scale={expon_scale_ttf}")
print(f"Weibull: shape={weibull_shape_ttf}, loc={weibull_loc_ttf}, scale={weibull_scale_ttf}")
print(f"Normal: mean={normal_mean_ttf}, std={normal_std_ttf}")
print(f"Lognormal: shape={lognormal_shape_ttf}, loc={lognormal_loc_ttf}, scale={lognormal_scale_ttf}")

=== Parameter Estimasi ===
Exponential: loc=0.0, scale=5607.734615384615
Weibull: shape=2.134115359328148, loc=0, scale=6323.080968149512
Normal: mean=5607.734615384615, std=2747.9564169470077
Lognormal: shape=0.6169841449853031, loc=0, scale=4810.781461997049


# SELECTED DISTRIBUTION

Jalankan salah satu distribusi terbaik dibawah ini.

## WEIBULL

In [ ]:
from scipy.special import gamma

# Parameter Weibull
beta = weibull_scale_ttf  # Scale parameter
k = weibull_shape_ttf    # Shape parameter

# Hitung MTTF
mttf_weibull = beta * gamma(1 + 1 / k)
print(f"Mean Time to Failure (MTTF) for Weibull Distribution: {mttf_weibull:.2f} jam")

# Reliability
reliability_at_t = np.exp(-(mttf_weibull / beta) ** k)
reliability_at_t = reliability_at_t * 100
prob_failure_at_t = 100 - reliability_at_t

print(f"Reliability pada saat t = {mttf_weibull:.2f} jam , adalah : {reliability_at_t:.2f}%")
print(f"artinya {prob_failure_at_t:.2f}% berpotensi failure")

Mean Time to Failure (MTTF) for Weibull Distribution: 5599.87 jam
Reliability pada saat t = 5599.87 jam , adalah : 46.22%
artinya 53.78% berpotensi failure


In [ ]:
# Parameter Weibull
k = weibull_shape_ttf     # Shape parameter
beta = weibull_scale_ttf  # Scale parameter

# Ambang reliabilitas (masukkan persen probabilitas yang diinginkan, semakin tinggi persen-nya semakin pendek durasi lifetime)
reliability_threshold = 0.75

# Cari waktu optimal
t = np.linspace(0, max(ttf_data), 10000)  # Rentang waktu
reliability = np.exp(-(t / beta) ** k)

# Waktu penggantian optimal
t_optimal = t[np.where(reliability <= reliability_threshold)[0][0]]

print(f"Waktu penggantian optimal berdasarkan reliabilitas > {reliability_threshold*100}%: {t_optimal:.2f} jam")

Waktu penggantian optimal berdasarkan reliabilitas > 75.0%: 3527.85 jam


## LOG-NORMAL

In [ ]:
from scipy.stats import lognorm

# Parameter yang sudah diestimasi
sigma = lognormal_shape_ttf  # σ (shape parameter)
scale = lognormal_scale_ttf   # e^μ (scale parameter)
mu = np.log(scale)           # μ = ln(scale)

# Hitung MTTF
mttf_log_normal = np.exp(mu + (sigma**2) / 2)

print(f"Mean Time to Failure (MTTF) for Log Normal Distribution: {mttf_log_normal:.2f} jam")

Mean Time to Failure (MTTF) for Log Normal Distribution: 5819.38 jam


In [ ]:
# Fungsi reliabilitas lognormal
z = (np.log(mttf_log_normal) - mu) / sigma
reliability_at_t = 1 - norm.cdf(z)
reliability_at_t = reliability_at_t * 100
prob_failure_at_t = 100 - reliability_at_t

print(f"Reliability pada saat t = {mttf_log_normal:.2f} jam , adalah : {reliability_at_t:.2f}%")
print(f"artinya {prob_failure_at_t:.2f}% berpotensi failure")

Reliability pada saat t = 5819.38 jam , adalah : 37.89%
artinya 62.11% berpotensi failure


## NORMAL

In [ ]:
mttf_normal = normal_mean_ttf
print(f"Mean Time to Failure (MTTF) for Normal Distribution: {mttf_normal:.2f} jam")

Mean Time to Failure (MTTF) for Normal Distribution: 8496.96 jam


## EXPONENTIAL

In [ ]:
# Eksponensial hanya punya parameter lambda = 1/scale
lambda_exp = 1 / expon_scale_ttf

# Hitung MTTF untuk distribusi eksponensial
mttf_expon = expon_scale_ttf  # MTTF = 1/lambda
print(f"Mean Time to Failure (MTTF) for Exponential Distribution: {mttf_expon:.2f} jam")

# Hitung Reliability pada t = MTTF
t = mttf_expon
reliability_at_t = np.exp(-lambda_exp * t) * 100  # dalam persen
prob_failure_at_t = 100 - reliability_at_t

# Tampilkan hasil
print(f"Reliability at t = {t:.2f} jam: {reliability_at_t:.2f}%")
print(f"Probability of Failure at t = {t:.2f} jam: {prob_failure_at_t:.2f}%")

Mean Time to Failure (MTTF) for Exponential Distribution: 5607.73 jam
Reliability at t = 5607.73 jam: 36.79%
Probability of Failure at t = 5607.73 jam: 63.21%
